## This code is written in Pytorch for the DISK project (NO Labels). 
@author: Sayantan
date: 29 September

In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import sys
import platform
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
import csv
import re ## For data manipulation

from PIL import Image, ImageOps

has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_built()
device = "mps" if torch.backends.mps.is_built() \
    else "cuda:0" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")

# print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")



Python Platform: macOS-13.6.2-arm64-arm-64bit
PyTorch Version: 2.1.0.dev20230623
GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('-f')
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--n_classes", type=int, default=10, help="number of classes for dataset")
parser.add_argument("--img_size", type=int, default=256, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=100, help="interval between image sampling")
opt = parser.parse_args()
print(opt)

Namespace(f='/Users/auddy/Library/Jupyter/runtime/kernel-5cf709b7-9b37-4a98-a55e-331d4c10716d.json', n_epochs=200, batch_size=64, lr=0.0002, b1=0.5, b2=0.999, n_cpu=8, latent_dim=100, n_classes=10, img_size=256, channels=1, sample_interval=100)


In [10]:
## Create a CSV file with Planet_Mass as labels and 
path = './'
Data_path = 'analysis_VAE_20JUlY2023/Disk_gas_plots/'
os.makedirs("analysis_data", exist_ok=True)
list_RT_path = glob.glob(path + Data_path + 'gas_gap*') ## make a list of all the RT folder where each folder is for each sim
list_sorted_RT_path  = sorted(list_RT_path, key=lambda f: [int(n) for n in re.findall(r"\d+", f)][-1]) ## sorting the images
df_images_folder =pd.DataFrame(list_sorted_RT_path,columns=["image_path"])
df_images_folder.to_csv(path+'./analysis_data/df_images_folder.csv')

# merging the csv with the simulation parameters files
joined_files = os.path.join("./analysis_VAE_20JUlY2023/", "Disk_gap_param*.csv")
  # A list of all joined files is returned
joined_list = glob.glob(joined_files)
# Finally, the files are joined
parameters_df = pd.concat(map(pd.read_csv, joined_list))
parameters_df['Planet_Mass'] = parameters_df['Planet_Mass'] / (3 * 10**-6) ## planet mass in Earth Mass units
# parameters_df
sorted_parmeters_df= parameters_df.sort_values('Sample#')
# # sorted_parmeters_df.to_csv(path+'./analysis_VAE_20JUlY2023/sorted_parmeters_df.csv')
complete_dataset = pd.concat([sorted_parmeters_df.reset_index(drop=True),df_images_folder],axis=1)
complete_dataset
complete_dataset.to_csv(path+'./analysis_data/complete_dataset.csv')
complete_dataset_mod = complete_dataset.drop(columns=['Sample#', 'SigmaSlope','Alpha'])

In [11]:
## Splitting the data for training and testing 
from sklearn.model_selection import train_test_split
split_1 = train_test_split(complete_dataset_mod, test_size=0.15, random_state=42)
(train, test) = split_1

In [12]:
## Defining a custom class that returns image tensor and the corresponding label for a given index

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,dataframe,transform = None):
        self.df = dataframe
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        filename = self.df["image_path"].iloc[index]
        label = self.df["Planet_Mass"].iloc[index]
        image = Image.open(filename)
        image = image.convert('L') ## Converting to gray-scale with one channel
#         print("THE SHAPE OF THE IMAGE",np.shape(image))
        left = 105
        top = 45
        right = 560
        bottom = 500 
        # Cropped image of above dimension" # (It will not change original image)
        image = image.crop((left, top, right, bottom))
        if self.transform is not None:
            image = self.transform(image)
#         print(np.shape(image))
        return image,label

In [13]:
transform_custom = transforms.Compose([transforms.Resize(opt.img_size),
   transforms.ToTensor()])
# image_output = CustomDataset(train,transform= transform_custom)
image_output = CustomDataset(train,transform = None)
image,label = image_output.__getitem__(4)
print(np.shape(image))
# gray_image = ImageOps.grayscale(image)
# print(type(gray_image))
# image.show()
# print(np.shape(gray_image))
# # print(image)
# gray_image.show()
# image.shape()

(455, 455)


In [14]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True}
#           'num_workers': 1}
# max_epochs = 2

In [15]:
def get_mean_std(loader):
    # Compute the mean and standard deviation of all pixels in the dataset
    num_pixels = 0
    mean = 0.0
    std = 0.0
    for images, _ in loader:
        batch_size, num_channels, height, width = images.shape
        num_pixels += batch_size * height * width
        mean += images.mean(axis=(0, 2, 3)).sum()
        std += images.std(axis=(0, 2, 3)).sum()

    mean /= num_pixels
    std /= num_pixels

    return mean, std

In [16]:
training_set = CustomDataset(train,transform=transform_custom)
training_generator = torch.utils.data.DataLoader(training_set, **params)
mean, std = get_mean_std(training_generator)

## This time normalizing the images
# transform_custom = transforms.Compose([transforms.Resize(32),
#    transforms.ToTensor(),transforms.Normalize(mean=mean, std=std)
# ])

# training_set = CustomDataset(train,transform=transform_custom)
# training_generator = torch.utils.data.DataLoader(training_set, **params)

In [17]:
# opt.channels, opt.img_size, opt.img_size = 1 ,32,32
img_shape = (opt.channels, opt.img_size, opt.img_size)
print(img_shape)

(1, 256, 256)


In [18]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity


In [19]:
# Loss functions
# adversarial_loss = torch.nn.MSELoss()
adversarial_loss = torch.nn.BCELoss()
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# if cuda:
generator.to(device)
discriminator.to(device)
adversarial_loss.to(device)
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [20]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(torch.FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))).to(device))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(torch.LongTensor(labels).to(device))
    gen_imgs = generator(z)
    print(np.shape(gen_imgs))
#     transform = transforms.Compose([transforms.ToPILImage(),transforms.Resize(32),transforms.ToTensor()])
#     gen_imgs = [transform(gen_imgs_) for gen_imgs_ in gen_imgs]
#     print(type(gen_imgs))
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)
# os.makedirs("images", exist_ok=True)
# sample_image(n_row=2, batches_done=20)

In [21]:
os.makedirs("images", exist_ok=True)
for epoch in range(opt.n_epochs):
    # Training
    for i, (imgs, _) in enumerate(training_generator):
        print(type(imgs))
        batch_size = imgs.shape[0]
        print("Batch_size",batch_size)
        # Adversarial ground truths
        valid = Variable(torch.FloatTensor(batch_size, 1).fill_(1.0).to(device), requires_grad=False)
        fake = Variable(torch.FloatTensor(batch_size, 1).fill_(0.0).to(device), requires_grad=False)
        
        # Configure input
        real_imgs = Variable(imgs.type(torch.FloatTensor).to(device))

#         print(real_imgs)
        
        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(torch.FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))).to(device))
#         gen_labels = Variable(torch.LongTensor(np.random.randint(0, opt.n_classes, batch_size)).to(device))

        # Generate a batch of images
        gen_imgs = generator(z)


         # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs)
        g_loss = adversarial_loss(validity, valid)


        g_loss.backward()
        optimizer_G.step()
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach())
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(training_generator), d_loss.item(), g_loss.item())
        )


        batches_done = epoch * len(training_generator) + i
#         print(epoch,batches_done,len(training_generator))
        print(np.shape(gen_imgs.data))
        if batches_done % opt.sample_interval == 0:
#             save_image(gen_imgs.data[:2], "images/%d.png" % batches_done, nrow=3, normalize=True)
            sample_image(n_row=5, batches_done=batches_done)


<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/200] [Batch 0/14] [D loss: 0.683843] [G loss: 0.668957]
torch.Size([64, 1, 256, 256])
torch.Size([25, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/200] [Batch 1/14] [D loss: 0.365741] [G loss: 0.656238]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/200] [Batch 2/14] [D loss: 0.366073] [G loss: 0.655661]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/200] [Batch 3/14] [D loss: 0.369094] [G loss: 0.650229]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/200] [Batch 4/14] [D loss: 0.358771] [G loss: 0.669715]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/200] [Batch 5/14] [D loss: 0.333605] [G loss: 0.720227]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/200] [Batch 6/14] [D loss: 0.288379] [G loss: 0.825269]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 0/

<class 'torch.Tensor'>
Batch_size 64
[Epoch 4/200] [Batch 6/14] [D loss: 1.375171] [G loss: 0.076097]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 4/200] [Batch 7/14] [D loss: 0.873285] [G loss: 0.200791]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 4/200] [Batch 8/14] [D loss: 0.696199] [G loss: 1.028578]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 4/200] [Batch 9/14] [D loss: 0.800693] [G loss: 0.234391]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 4/200] [Batch 10/14] [D loss: 0.638432] [G loss: 0.335425]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 4/200] [Batch 11/14] [D loss: 0.480910] [G loss: 1.231130]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 4/200] [Batch 12/14] [D loss: 0.546945] [G loss: 0.419330]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 18
[Epoch 4/200] [Batch 13/14] [D loss:

<class 'torch.Tensor'>
Batch_size 64
[Epoch 8/200] [Batch 12/14] [D loss: 0.498718] [G loss: 0.628473]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 18
[Epoch 8/200] [Batch 13/14] [D loss: 0.555324] [G loss: 0.488842]
torch.Size([18, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 9/200] [Batch 0/14] [D loss: 0.597476] [G loss: 0.507733]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 9/200] [Batch 1/14] [D loss: 0.657672] [G loss: 0.352962]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 9/200] [Batch 2/14] [D loss: 0.713993] [G loss: 0.559994]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 9/200] [Batch 3/14] [D loss: 0.970905] [G loss: 0.158488]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 9/200] [Batch 4/14] [D loss: 0.654332] [G loss: 0.345174]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 9/200] [Batch 5/14] [D loss: 1

<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 4/14] [D loss: 0.610058] [G loss: 0.577437]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 5/14] [D loss: 0.594913] [G loss: 0.646775]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 6/14] [D loss: 0.581304] [G loss: 0.566384]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 7/14] [D loss: 0.569550] [G loss: 0.723830]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 8/14] [D loss: 0.560974] [G loss: 0.578439]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 9/14] [D loss: 0.565532] [G loss: 0.805945]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 10/14] [D loss: 0.614571] [G loss: 0.434062]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 13/200] [Batch 11/14] [D

<class 'torch.Tensor'>
Batch_size 64
[Epoch 17/200] [Batch 10/14] [D loss: 0.712024] [G loss: 0.355179]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 17/200] [Batch 11/14] [D loss: 0.909299] [G loss: 1.188722]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 17/200] [Batch 12/14] [D loss: 0.659170] [G loss: 0.455329]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 18
[Epoch 17/200] [Batch 13/14] [D loss: 0.615593] [G loss: 0.509990]
torch.Size([18, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 18/200] [Batch 0/14] [D loss: 0.584642] [G loss: 0.801888]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 18/200] [Batch 1/14] [D loss: 0.578486] [G loss: 0.615078]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 18/200] [Batch 2/14] [D loss: 0.623967] [G loss: 0.672813]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 18/200] [Batch 3/14] 

...

<class 'torch.Tensor'>
Batch_size 64
[Epoch 180/200] [Batch 7/14] [D loss: 0.677857] [G loss: 0.726540]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 180/200] [Batch 8/14] [D loss: 0.622607] [G loss: 0.725588]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 180/200] [Batch 9/14] [D loss: 0.567701] [G loss: 0.773638]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 180/200] [Batch 10/14] [D loss: 0.560429] [G loss: 1.004828]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 180/200] [Batch 11/14] [D loss: 0.595417] [G loss: 0.668033]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 180/200] [Batch 12/14] [D loss: 0.606274] [G loss: 0.913972]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 18
[Epoch 180/200] [Batch 13/14] [D loss: 0.676113] [G loss: 0.571598]
torch.Size([18, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 181/200] [Batc

<class 'torch.Tensor'>
Batch_size 18
[Epoch 184/200] [Batch 13/14] [D loss: 0.701813] [G loss: 0.705912]
torch.Size([18, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 185/200] [Batch 0/14] [D loss: 0.721314] [G loss: 0.461957]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 185/200] [Batch 1/14] [D loss: 0.701862] [G loss: 0.549642]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 185/200] [Batch 2/14] [D loss: 0.684606] [G loss: 0.746777]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 185/200] [Batch 3/14] [D loss: 0.682637] [G loss: 0.718691]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 185/200] [Batch 4/14] [D loss: 0.669413] [G loss: 0.617091]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 185/200] [Batch 5/14] [D loss: 0.657955] [G loss: 0.676636]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 185/200] [Batch 6

<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 4/14] [D loss: 0.689639] [G loss: 0.948249]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 5/14] [D loss: 0.624154] [G loss: 0.641041]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 6/14] [D loss: 0.555562] [G loss: 0.883519]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 7/14] [D loss: 0.539043] [G loss: 0.999740]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 8/14] [D loss: 0.572903] [G loss: 0.773180]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 9/14] [D loss: 0.576076] [G loss: 0.751475]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 10/14] [D loss: 0.614383] [G loss: 0.847587]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 189/200] [Batch 1

<class 'torch.Tensor'>
Batch_size 64
[Epoch 193/200] [Batch 9/14] [D loss: 0.489871] [G loss: 0.981423]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 193/200] [Batch 10/14] [D loss: 0.550773] [G loss: 0.713951]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 193/200] [Batch 11/14] [D loss: 0.695694] [G loss: 1.176904]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 193/200] [Batch 12/14] [D loss: 1.140575] [G loss: 0.134174]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 18
[Epoch 193/200] [Batch 13/14] [D loss: 1.202838] [G loss: 1.447173]
torch.Size([18, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 194/200] [Batch 0/14] [D loss: 0.940034] [G loss: 0.244468]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 194/200] [Batch 1/14] [D loss: 0.798355] [G loss: 0.422286]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 194/200] [Batc

<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 0/14] [D loss: 0.730862] [G loss: 0.421517]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 1/14] [D loss: 0.886992] [G loss: 1.201715]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 2/14] [D loss: 1.084162] [G loss: 0.148217]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 3/14] [D loss: 0.829578] [G loss: 0.785100]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 4/14] [D loss: 0.772581] [G loss: 0.640553]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 5/14] [D loss: 0.736062] [G loss: 0.491519]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 6/14] [D loss: 0.677919] [G loss: 0.610687]
torch.Size([64, 1, 256, 256])
<class 'torch.Tensor'>
Batch_size 64
[Epoch 198/200] [Batch 7/

In [ ]:
# def sample_image(n_row, batches_done):
#     """Saves a grid of generated digits ranging from 0 to n_classes"""
#     # Sample noise
#     z = Variable(torch.FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))).to(device))
#     # Get labels ranging from 0 to n_classes for n rows
# #     labels = np.array([num for _ in range(n_row) for num in range(n_row)])
# #     labels = Variable(torch.LongTensor(labels).to(device))
#     gen_imgs = generator(z)
#     print(np.shape(gen_imgs))
#     save_image(gen_imgs.data, "images/test%d.png" % batches_done, nrow=n_row, normalize=True)
# os.makedirs("images", exist_ok=True)
# sample_image(n_row=3, batches_done=20)